In [1]:
import snman
from snman.constants import *

PERIMETER = '_debug'

# Set these paths according to your own setup
path = 'C:/Users/lballo/polybox/RESEARCH/SNMan/SNMan Shared/data_v2/varia/export_for_storymap/'
inputs_path = path + 'inputs/'
outputs_path = path + 'outputs/'

CRS_internal = 2056      # for Zurich
CRS_for_export = 4326

C:\Users\lballo\AppData\Roaming\Python\Python39\site-packages\osmnx\utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


In [2]:
print('Load street graph')
G = snman.io.load_street_graph(
    inputs_path + 'street_graph_edges.gpkg',
    inputs_path + 'street_graph_nodes.gpkg',
    crs=CRS_internal
)

Load street graph


In [3]:
for uvk, data in G.edges.items():
    print(data[KEY_LANES_DESCRIPTION])

['H>', 'H>', 'H>', 'H>', 'H>']
['H>', 'H>', 'H>', 'H>']
['H<', 'H<', 'H<', 'H<']
['H<', 'H<', 'H<', 'H<']
['H<']
['H>']
['H<', 'H<', 'H<']
['H>', 'H>', 'H>']
['X-']
['X-']
['H>', 'H>', 'H>']
['H<', 'H<', 'H<']
['H<', 'H<']
['H>', 'H>', 'H>']
['H<']
['L<', 'M<', 'M<', 'M<', 'M>']
['X-', 'M<', 'M>', 'L>']
['L<', 'M<', 'M>']
['M<', 'M<', 'M>', 'L>']
['L<', 'M-', 'L>']
['M<', 'M>']
['R-', 'L<', 'M-', 'L>']
['L<', 'M<', 'M>', 'L>']
['R-', 'L<', 'M-']
['M<', 'M>', 'L>']
['L<', 'M<', 'M>']
['L<', 'M<', 'T<', 'M>', 'L>']
['T<', 'M<', 'M>', 'L>']
['M-', 'R-', 'R-', 'R-']
['M-']
['M-', 'R-']
['R-', 'R-', 'R-', 'M-']
['R-', 'R-', 'R-', 'M-']
['S<', 'M>', 'R-']
['M-', 'R-']
['S<', 'M>', 'R-']
['R-', 'M-']
['H<', 'H<', 'H<']
['H>', 'H>']
['H>']
['H>', 'H>']
['H>', 'H>', 'H>']
['H<']
['H<', 'H<']
['H>', 'H>']
['H>', 'H>']
['L<', 'M<', 'M>', 'L>']
['M<', 'M>']
['H<', 'H<']
['H>', 'H>']
['H>']
['H<', 'H<']
['H>', 'H>', 'H>']
['H<', 'H<', 'H<']
['M<', 'M<', 'M>', 'M>', 'T<', 'T>', 'M<', 'M<']
['R-', 'T

In [4]:
for uvk, data in G.edges.items():
    data['length'] = 0

In [5]:
# RUN THIS FOR "AFTER_TRANSIT"

if 0:
    state = 'after_transit'
    lanes_attribute = KEY_GIVEN_LANES_DESCRIPTION
    
    import copy
    
    #G = snman.street_graph.filter_lanes_by_modes(G, {MODE_CYCLING, MODE_TRANSIT, MODE_PRIVATE_CARS, MODE_FOOT})
    
    snman.distribution.set_given_lanes(G)
    
    # remove all edges with no transit
    H = copy.deepcopy(G)
    for uvk, edge in H.edges.items():
        if edge.get('pt_bus') is False:
            G.remove_edge(*uvk)
    
    for uvk, edge in G.edges.items():
        # copy transit lanes from 'before'
        transit_lanes = [
            lane for lane in edge.get(KEY_LANES_DESCRIPTION) 
            if set(snman.space_allocation._lane_properties(lane).modes) == {MODE_TRANSIT}
        ]
        # add all given motorized lanes
        edge[lanes_attribute] = edge.get(lanes_attribute, []) + transit_lanes
        # keep only those given lanes that are fixed
        edge[lanes_attribute] = [
            lane for lane in edge[lanes_attribute]
            if snman.space_allocation._lane_properties(lane).direction
               in {DIRECTION_FORWARD, DIRECTION_BACKWARD, DIRECTION_BOTH}
        ]

In [6]:
# RUN THIS FOR "AFTER_MOTORIZED"
if 0:
    state = 'after_motorized'
    lanes_attribute = KEY_LANES_DESCRIPTION_AFTER
    
    G = snman.street_graph.filter_lanes_by_modes(G, {MODE_TRANSIT, MODE_PRIVATE_CARS}, lane_description_key=lanes_attribute)

In [7]:
# RUN THIS FOR "BEFORE"
if 0:
    state = 'before'
    lanes_attribute = KEY_LANES_DESCRIPTION

In [8]:
# RUN THIS FOR "AFTER WITHOUT PARKING"
if 1:
    state = 'after_without_parking'
    lanes_attribute = KEY_LANES_DESCRIPTION_AFTER
    G = snman.street_graph.filter_lanes_by_modes(G, {MODE_TRANSIT, MODE_PRIVATE_CARS, MODE_CYCLING}, lane_description_key=lanes_attribute)

In [9]:
# RUN THIS FOR "AFTER"
if 0:
    state = 'after'
    lanes_attribute = KEY_LANES_DESCRIPTION_AFTER

In [10]:
snman.space_allocation.reorder_lanes(G, lanes_attribute=lanes_attribute)
snman.space_allocation.normalize_cycling_lanes(G, lanes_key=lanes_attribute)

In [11]:
format = 'geojson'
#format = 'gpkg'
scaling = 1

path_axes = outputs_path + f'lanes_axes.{format}'
path_arrows = outputs_path + f'lanes_arrows.{format}'
path_polygons = outputs_path + f'lanes_polygons.{format}'
path_separators = outputs_path + f'lanes_separators.{format}'

In [12]:
import shapely
import copy

L = snman.lane_graph.create_lane_graph(G, lanes_attribute=lanes_attribute)
#snman.io.export_lane_geometries(L, process_path + 'lanes.gpkg')

_, edges = snman.oxc.graph_to_gdfs(L)
edges = edges.query("lanetype!='S'")

# apply offset
edges_axes = edges.apply(
    lambda edge:
        shapely.offset_curve(
            edge['geometry'],
            -edge['horizontal_position'] * scaling * (-1 if edge['backward']==1 else 1)
        ),
        axis=1
)

edges.geometry = edges_axes

In [13]:
# create arrows
edges_arrows = edges.apply(
    lambda edge:
        shapely.intersection(
            shapely.buffer(
                edge['geometry'].interpolate(0.5, normalized=True),
                3
            ),
            edge['geometry']
        ),
        axis=1
)

edges.geometry = edges_arrows
edges_filtered = edges.query("lanetype != 'R'")
snman.io.export_gdf(edges_filtered, path_arrows)

In [14]:
edges = edges.query("instance==1")

# apply buffer
edges.geometry = edges_axes
edges_polygons = edges.apply(
    lambda edge:
        shapely.buffer(
            edge['geometry'],
            edge['width']/2 * scaling, cap_style='flat'
        ),
        axis=1
)

def fn(edge):
    try:
        return shapely.MultiLineString([
            shapely.offset_curve(
                edge['geometry'],
                edge['width']/2 * scaling * (-1 if edge['backward']==1 else 1)
            ),
            shapely.offset_curve(
                edge['geometry'],
                -edge['width']/2 * scaling * (-1 if edge['backward']==1 else 1)
            )
        ])
    except (shapely.errors.EmptyPartError):
        return shapely.MultiLineString()
    

# create separators
edges_separators = edges.apply(fn, axis=1)


edges.geometry = edges_polygons
snman.io.export_gdf(edges, path_polygons)

edges.geometry = edges_separators
snman.io.export_gdf(edges, path_separators)